In [148]:

!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-core --progress-bar off
!pip -qqq install langchain-community --progress-bar off

!pip -qqq install langchain-groq --progress-bar off
!pip -qqq install crewai==0.64.0 --progress-bar off
!pip -qqq install crewai-tools --progress-bar off
#!pip -qqq install 'crewai[tools]' --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai-tools 0.14.0 requires langchain>=0.3.1, but you have langchain 0.2.17 which is incompatible.
langchain-groq 0.2.1 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.2.43 which is incompatible.
langchain-huggingface 0.1.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.2.43 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.64.0 requires langchain<0.3.0,>=0.2.16, but you have langchain 0.3.7 which is incompatible.


In [149]:
#imports
import json
import os
import sqlite3
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union

import pandas as pd

from crewai import Agent, Crew, Process, Task
from crewai_tools import tool
from google.colab import userdata
from langchain.schema import AgentFinish
from langchain.schema.output import LLMResult
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)

from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


In [150]:
!pip install --upgrade langchain-core
!pip install --upgrade langchain-community

In [151]:
!pip install unstructured

In [152]:
!pip install langchain-huggingface
!pip install langchain-chroma

In [153]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [160]:



loader = TextLoader("documents/gen_info.txt")
docs= loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=30,
    length_function=len,
    add_start_index=True,
)
chunks = text_splitter.split_documents(docs)


In [162]:
chunks[10]

Document(metadata={'source': 'documents/gen_info.txt', 'start_index': 770}, page_content='Sickle cell anemia: A genetic condition where RBCs have an abnormal shape, affecting their function.')

In [163]:
print(f"split into {len(chunks)} chunks")

split into 73 chunks


In [164]:
embeddings= HuggingFaceEmbeddings()

In [165]:
db2= Chroma.from_documents(chunks, embeddings)

In [166]:
persist_directory = "chroma_db"  # Choose a directory name
db2 = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


In [167]:
db2.add_documents(chunks)

['644ca687-4e4e-436e-80a6-255326d3b77b',
 '6eab5501-9e2c-439f-a448-9b2333e9100e',
 'c65bc0e8-52c6-4ff7-a61c-6b67ec12ca57',
 'f7e053bb-fe38-47be-be9c-d696c2d386b8',
 '24e7152f-ec40-4ebb-bf45-ebfa21c1f309',
 '7cbfbfb6-6f1e-4d9f-8e6f-d8f16f57bf03',
 '714acfcf-94a4-477a-9d2c-a704fe0e44b6',
 '570095ad-8746-4b50-a912-49cc60198915',
 '4f0ef1c0-c134-4930-b794-f5aaa670715e',
 '6b5eb908-4b56-4c46-a63b-f0d9507e8696',
 '231d5dbc-612e-49e5-a1cd-9316a6e1e868',
 '1809e0f2-34a3-4781-962a-45fe1116e5cb',
 '8d100754-5450-43b4-b69c-2cda1990294b',
 '2c934a51-d543-42c6-88c1-968462fced39',
 'f0929359-786a-47e9-8714-5be8ec24c369',
 '839b474e-e6f0-4a82-b397-8cfa75409de4',
 'bff5e61a-dca2-4013-9c74-7624d7f6637d',
 'e84c71a1-da00-4d31-a0df-ec8f08f4633f',
 '6389e8e6-0cdc-44fb-b6b8-2c0b276c41f3',
 '02af2516-139d-4f6f-87c0-3721c7bb0f1f',
 'f53f7fac-eb33-4d68-934e-91e3ca8218b5',
 '2204bc3b-972f-49c0-b6e8-59e540f39e85',
 '6197738c-f008-432b-bf82-fac7e753f594',
 'f2ad80b6-e81f-4ea0-89ae-89cd062cee22',
 'ba6b55ad-ea40-

In [168]:
#set api key for llm
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [169]:
#read csv
df = pd.read_csv("patient_vitals.csv")
df.head()

,id,Age,Bgr,Al,Bu,Sc,Sod,Pot,Hemo,Pcv,Wc,Rc,Htn,Dm,Cad,Pe,Ane,Chloride,BiCarbonate,date
0,11052880,21,86,3,120,9.6,138,3.90,8.0,4.5,4.5,2.5,no,no,no,no,yes,97.0,22,19-Oct-23
1,14687354,33,110,3,114,9.4,140,4.40,10.5,2.5,4.8,2.8,no,no,no,no,yes,98.4,24,17-Aug-21
2,11523733,60,125,3,110,8.6,148,4.20,7.0,2.1,4.0,2.4,yes,no,no,yes,yes,99.0,21,23-Feb-21
3,19561133,35,110,3,148,11.5,140,4.10,10.5,2.7,5.1,3.1,yes,no,no,no,yes,99.0,24,14-Jan-22
4,11523733,60,110,3,128,10.8,145,3.99,7.5,2.2,4.8,3.2,yes,yes,yes,no,yes,98.0,24,18-Sep-22


In [170]:
#put dataframe into sql db
connection = sqlite3.connect("vitals.db")
df.to_sql(name="vitals", con=connection)

ValueError: Table 'vitals' already exists.

In [171]:
#logging mechanism

# to store information about an event
@dataclass
class Event:
    event: str
    timestamp: str
    text: str

#returns the current time
def _current_time() -> str:
    return datetime.now(timezone.utc).isoformat()

#handles logging events when the LLM starts and ends.
class LLMCallbackHandler(BaseCallbackHandler):
    def __init__(self, log_path: Path):
        self.log_path = log_path
#called when the LLM starts running
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        """Run when LLM starts running."""
        assert len(prompts) == 1
        event = Event(event="llm_start", timestamp=_current_time(), text=prompts[0])
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")
#called when the LLM ends
    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        generation = response.generations[-1][-1].message.content
        event = Event(event="llm_end", timestamp=_current_time(), text=generation)
        with self.log_path.open("a", encoding="utf-8") as file:
            file.write(json.dumps(asdict(event)) + "\n")

In [172]:
#initialize model
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    callbacks=[LLMCallbackHandler(Path("prompts.jsonl"))],
)

In [173]:
#testing
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("human", human)])

chain = prompt | llm
response = chain.invoke(
    {
        "text": "explain large language models"
    }
)

print(response.content)

Large language models are a type of artificial intelligence (AI) designed to process and understand human language. They are trained on vast amounts of text data and can generate human-like language outputs, making them incredibly powerful and versatile tools. Here's a detailed explanation of large language models:

**What are large language models?**

Large language models are neural networks that are trained on massive datasets of text to learn the patterns, structures, and relationships within language. They are designed to capture the complexities of human language, including grammar, syntax, semantics, and pragmatics.

**How do they work?**

Large language models work by using a type of recurrent neural network (RNN) called a transformer. The transformer architecture was introduced in a research paper by Vaswani et al. in 2017 and has since become the standard for natural language processing (NLP) tasks.

Here's a high-level overview of how large language models work:

1. **Input*

In [176]:
#connect db
db= SQLDatabase.from_uri("sqlite:///vitals.db")

In [177]:
#tool to list tables
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

In [178]:
list_tables.run()

Using Tool: list_tables


'vitals'

In [179]:
#get schema tool
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

In [180]:
print(tables_schema.run("vitals"))

Using Tool: tables_schema

CREATE TABLE vitals (
	"index" INTEGER, 
	id INTEGER, 
	"Age" INTEGER, 
	"Bgr" INTEGER, 
	"Al" INTEGER, 
	"Bu" INTEGER, 
	"Sc" REAL, 
	"Sod" INTEGER, 
	"Pot" REAL, 
	"Hemo" REAL, 
	"Pcv" REAL, 
	"Wc" REAL, 
	"Rc" REAL, 
	"Htn" TEXT, 
	"Dm" TEXT, 
	"Cad" TEXT, 
	"Pe" TEXT, 
	"Ane" TEXT, 
	"Chloride" REAL, 
	"BiCarbonate" INTEGER, 
	date TEXT
)

/*
3 rows from vitals table:
index	id	Age	Bgr	Al	Bu	Sc	Sod	Pot	Hemo	Pcv	Wc	Rc	Htn	Dm	Cad	Pe	Ane	Chloride	BiCarbonate	date
0	11052880	21	86	3	120	9.6	138	3.9	8.0	4.5	4.5	2.5	no	no	no	no	yes	97.0	22	19-Oct-23
1	14687354	33	110	3	114	9.4	140	4.4	10.5	2.5	4.8	2.8	no	no	no	no	yes	98.4	24	17-Aug-21
2	11523733	60	125	3	110	8.6	148	4.2	7.0	2.1	4.0	2.4	yes	no	no	yes	yes	99.0	21	23-Feb-21
*/


In [181]:
#query tool
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

In [182]:

execute_sql.run("SELECT * FROM vitals where Hemo>5 Limit 5")

Using Tool: execute_sql


"[(0, 11052880, 21, 86, 3, 120, 9.6, 138, 3.9, 8.0, 4.5, 4.5, 2.5, 'no', 'no', 'no', 'no', 'yes', 97.0, 22, '19-Oct-23'), (1, 14687354, 33, 110, 3, 114, 9.4, 140, 4.4, 10.5, 2.5, 4.8, 2.8, 'no', 'no', 'no', 'no', 'yes', 98.4, 24, '17-Aug-21'), (2, 11523733, 60, 125, 3, 110, 8.6, 148, 4.2, 7.0, 2.1, 4.0, 2.4, 'yes', 'no', 'no', 'yes', 'yes', 99.0, 21, '23-Feb-21'), (3, 19561133, 35, 110, 3, 148, 11.5, 140, 4.1, 10.5, 2.7, 5.1, 3.1, 'yes', 'no', 'no', 'no', 'yes', 99.0, 24, '14-Jan-22'), (4, 11523733, 60, 110, 3, 128, 10.8, 145, 3.99, 7.5, 2.2, 4.8, 3.2, 'yes', 'yes', 'yes', 'no', 'yes', 98.0, 24, '18-Sep-22')]"

In [183]:
#query check tool
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

In [184]:

check_sql.run("SELECT * WHERE salary > 10000 LIMIT 5 table = salaries")

Using Tool: check_sql


'SELECT * FROM salaries WHERE salary > 10000 LIMIT 5'

In [185]:
#query writing agent
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



In [186]:
#analyst agent
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
        your insigts are accurate and based on the data.
    """
    ),
    llm=llm,
    allow_delegation=False,
)


ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



In [187]:
#medical report agent
report_writer = Agent(
    role="Senior Medical nurse report writer",
    goal="Write an concise and accurate Medical answer based on the work of the analyst",
    backstory=dedent(
        """
        Your writing style is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        You always answer in a patient friendly way with less technical terms
        """
    ),
    llm=llm,
    allow_delegation=False,
)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



In [188]:
extract_data = Task(
    description="Extract data that is required for the query :{query}. Try to Limit the number of entires unless it could lead to a worse answer. Make sure to select only the column required for the query.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [189]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis relavent to the :{query}. Pay close attention about what value corresponds to what vital to avoid mixing up your vitals and values",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

In [190]:
write_report = Task(
    description=dedent(
        """
        Provide a concise and accurate medical response based on the analysis in second person point of view adressing the patient. The response should be tailored to answer the patient's question: {query}. If the patient asks for a summary make sure you sumamrize the relavent information. make sure the response is less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

In [191]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=True,
    memory=False,
    output_log_file="crew.log",
)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



In [192]:

inputs = {
    "query": "summay of patient 17017753 Hemo levels"
}

result = crew.kickoff(inputs=inputs)

ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=llama3-70b-8192
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers
ERROR:root:Failed to get supported params: argument of type 'NoneType' is not iterable
ERROR:root:LiteLLM call failed: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=llama3-70b-8192
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `co


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

# Agent: Senior Database Developer
## Task: Extract data that is required for the query :summay of patient 17017753 Hemo levels. Try to Limit the number of entires unless it could lead to a worse answer. Make sure to select only the column required for the query.

Provider List: https://docs.litellm.ai/docs/providers

# Agent: Senior Database Developer
## Task: Extract data that is required for the query :summay of patient 17017753 Hemo levels. Try to Limit the number of entires unless it could lead to a worse answer. Make sure to select only the column required for the query.

Provider List: https://docs.litellm.ai/docs/providers

# Agent: Senior Database Developer
## Task: Extract data that is required for the query :summay of patient 17017753 Hemo levels. Try to 

BadRequestError: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=llama3-70b-8192
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers

In [ ]:
print(result)

**Executive Medical Summary: Patient 17017753 Hemo Levels**

**Patient ID:** 17017753
**Hemo Levels Summary:**

* **Highest Recorded:** 10.5 (14-Jan-22)
* **Lowest Recorded:** 9.5 (23-Feb-21, 17-Aug-21, 16-Jan-23)
* **Fluctuation:** Hemo levels have fluctuated over time, requiring close monitoring to identify changes or trends.

**Recommendation:** Closely monitor Hemo levels to ensure prompt identification and addressing of any changes or trends, enabling informed decisions about patient care and treatment.


In [ ]:
question="summarize my hemo levels"
id="17017753"

In [ ]:
template= "question from patient id {} : {}".format(id,question)
print(template)

question from patient id 17017753 : summarize my hemo levels


In [ ]:
inputs = {
    "query": template
}

result = crew.kickoff(inputs=inputs)

NameError: name 'crew' is not defined

In [ ]:
question="when did i have the highest hemoglobin levels"
id="17017753"


In [ ]:
template= "question from patient id {} : {}".format(id,question)
print(template)

question from patient id 17017753 : summarize my hemo levels


In [ ]:
inputs = {
    "query": template
}

result = crew.kickoff(inputs=inputs)

 [2024-08-14 23:32:40][DEBUG]: == Working Agent: Senior Database Developer
 [2024-08-14 23:32:40][INFO]: == Starting Task: Extract data that is required for the query :question from patient id 17017753 : when did i have the highest hemoglobin levels. Take 5 entires and make sure to select only the column required for the query. 
 

vitals

 


CREATE TABLE vitals (
	"index" INTEGER, 
	id INTEGER, 
	"Age" INTEGER, 
	"Bgr" INTEGER, 
	"Al" INTEGER, 
	"Bu" INTEGER, 
	"Sc" REAL, 
	"Sod" INTEGER, 
	"Pot" REAL, 
	"Hemo" REAL, 
	"Pcv" REAL, 
	"Wc" REAL, 
	"Rc" REAL, 
	"Htn" TEXT, 
	"Dm" TEXT, 
	"Cad" TEXT, 
	"Pe" TEXT, 
	"Ane" TEXT, 
	"Chloride" REAL, 
	"BiCarbonate" INTEGER, 
	date TEXT
)

/*
3 rows from vitals table:
index	id	Age	Bgr	Al	Bu	Sc	Sod	Pot	Hemo	Pcv	Wc	Rc	Htn	Dm	Cad	Pe	Ane	Chloride	BiCarbonate	date
0	11052880	21	86	3	120	9.6	138	3.9	8.0	4.5	4.5	2.5	no	no	no	no	yes	97.0	22	19-Oct-23
1	14687354	33	110	3	114	9.4	140	4.4	10.5	2.5	4.8	2.8	no	no	no	no	yes	98.4	24	17-Aug-21
2	11523733	60	

In [ ]:
print(result)

**Executive Medical Summary**

**Patient ID:** 17017753
**Highest Hemoglobin Levels:**

* **Date:** 14-Jan-22
* **Hemoglobin Value:** 10.5


In [ ]:
question="when did i have the highest hemoglobin levels"
id="17017753"
geninfo_rag= db2.similarity_search("when did i have the highest hemoglobin levels", k=1)
template= "question from patient id {} : {}. Some general info that may or may not be useful:{}".format(id,question,geninfo_rag[0].page_content)
print(template)

question from patient id 17017753 : when did i have the highest hemoglobin levels. Some general info that may or may not be useful:A patient is considered to be Anemic if his hemoglobin (Hemo) levels are greater than 10.


In [ ]:
inputs = {
    "query": template
}

result = crew.kickoff(inputs=inputs)

 [2024-08-15 00:56:54][DEBUG]: == Working Agent: Senior Database Developer
 [2024-08-15 00:56:54][INFO]: == Starting Task: Extract data that is required for the query :question from patient id 17017753 : when did i have the highest hemoglobin levels. Some general info that may or may not be useful:A patient is considered to be Anemic if his hemoglobin (Hemo) levels are greater than 10.. Take 5 entires and make sure to select only the column required for the query. 
 

vitals

 


CREATE TABLE vitals (
	"index" INTEGER, 
	id INTEGER, 
	"Age" INTEGER, 
	"Bgr" INTEGER, 
	"Al" INTEGER, 
	"Bu" INTEGER, 
	"Sc" REAL, 
	"Sod" INTEGER, 
	"Pot" REAL, 
	"Hemo" REAL, 
	"Pcv" REAL, 
	"Wc" REAL, 
	"Rc" REAL, 
	"Htn" TEXT, 
	"Dm" TEXT, 
	"Cad" TEXT, 
	"Pe" TEXT, 
	"Ane" TEXT, 
	"Chloride" REAL, 
	"BiCarbonate" INTEGER, 
	date TEXT
)

/*
3 rows from vitals table:
index	id	Age	Bgr	Al	Bu	Sc	Sod	Pot	Hemo	Pcv	Wc	Rc	Htn	Dm	Cad	Pe	Ane	Chloride	BiCarbonate	date
0	11052880	21	86	3	120	9.6	138	3.9	8.0	4.5	4.5	

In [ ]:
question="am I Anemic"
id="17017753"
geninfo_rag= db2.similarity_search(question, k=1)
template= "question from patient id {} : {}. Some general info that may or may not be useful:{}".format(id,question,geninfo_rag[0].page_content)
print(template)

question from patient id 17017753 : am I Anemic. Some general info that may or may not be useful:A patient is considered to be Anemic if his hemoglobin (Hemo) levels are greater than 10.


In [ ]:
inputs = {
    "query": template
}

result = crew.kickoff(inputs=inputs)

 [2024-08-15 01:12:27][DEBUG]: == Working Agent: Senior Database Developer
 [2024-08-15 01:12:27][INFO]: == Starting Task: Extract data that is required for the query :question from patient id 17017753 : am I Anemic. Some general info that may or may not be useful:A patient is considered to be Anemic if his hemoglobin (Hemo) levels are greater than 10.. Try to Limit the number of entires if possible and make sure to select only the column required for the query.
 

vitals

 


CREATE TABLE vitals (
	"index" INTEGER, 
	id INTEGER, 
	"Age" INTEGER, 
	"Bgr" INTEGER, 
	"Al" INTEGER, 
	"Bu" INTEGER, 
	"Sc" REAL, 
	"Sod" INTEGER, 
	"Pot" REAL, 
	"Hemo" REAL, 
	"Pcv" REAL, 
	"Wc" REAL, 
	"Rc" REAL, 
	"Htn" TEXT, 
	"Dm" TEXT, 
	"Cad" TEXT, 
	"Pe" TEXT, 
	"Ane" TEXT, 
	"Chloride" REAL, 
	"BiCarbonate" INTEGER, 
	date TEXT
)

/*
3 rows from vitals table:
index	id	Age	Bgr	Al	Bu	Sc	Sod	Pot	Hemo	Pcv	Wc	Rc	Htn	Dm	Cad	Pe	Ane	Chloride	BiCarbonate	date
0	11052880	21	86	3	120	9.6	138	3.9	8.0	4.5	4.5	2.5

In [ ]:
question="when did i have the highest hemoglobin readings"
id="17017753"
geninfo_rag= db2.similarity_search(question, k=1)
template= "question from patient id {} : {}. Some general info that may or may not be useful:{}".format(id,question,geninfo_rag[0].page_content)
print(template)

question from patient id 17017753 : when did i have the highest hemoglobin readings. Some general info:A patient is considered to be Anemic if his hemoglobin (Hemo) levels are greater than or equal to


In [ ]:
inputs = {
    "query": template
}

result = crew.kickoff(inputs=inputs)

 [2024-08-15 01:28:32][DEBUG]: == Working Agent: Senior Database Developer
 [2024-08-15 01:28:32][INFO]: == Starting Task: Extract data that is required for the query :question from patient id 17017753 : when did i have the highest hemoglobin readings. Some general info that may or may not be useful:A patient is considered to be Anemic if his hemoglobin (Hemo) levels are greater than or equal to. Try to Limit the number of entires unless it could lead to a worse answer. Make sure to select only the column required for the query.
 

vitals

 


CREATE TABLE vitals (
	"index" INTEGER, 
	id INTEGER, 
	"Age" INTEGER, 
	"Bgr" INTEGER, 
	"Al" INTEGER, 
	"Bu" INTEGER, 
	"Sc" REAL, 
	"Sod" INTEGER, 
	"Pot" REAL, 
	"Hemo" REAL, 
	"Pcv" REAL, 
	"Wc" REAL, 
	"Rc" REAL, 
	"Htn" TEXT, 
	"Dm" TEXT, 
	"Cad" TEXT, 
	"Pe" TEXT, 
	"Ane" TEXT, 
	"Chloride" REAL, 
	"BiCarbonate" INTEGER, 
	date TEXT
)

/*
3 rows from vitals table:
index	id	Age	Bgr	Al	Bu	Sc	Sod	Pot	Hemo	Pcv	Wc	Rc	Htn	Dm	Cad	Pe	Ane	Chloride

In [ ]:
question="what were my hemo levels between august 1st 2021 and may 1st 2022 "
id="17017753"
geninfo_rag= db2.similarity_search(question, k=1)
template= "question from patient id {} : {}. Some general info that may or may not be useful:{}".format(id,question,geninfo_rag[0].page_content)
print(template)

question from patient id 17017753 : what were my hemo levels between august 1st 2021 and may 1st 2022 . Some general info that may or may not be useful:A patient is considered to be Anemic if his hemoglobin (Hemo) levels are greater than or equal to


In [ ]:
inputs = {
    "query": template
}

result = crew.kickoff(inputs=inputs)

 [2024-08-15 01:46:32][DEBUG]: == Working Agent: Senior Database Developer
 [2024-08-15 01:46:32][INFO]: == Starting Task: Extract data that is required for the query :question from patient id 17017753 : what were my hemo levels between august 1st 2021 and may 1st 2022 . Some general info that may or may not be useful:A patient is considered to be Anemic if his hemoglobin (Hemo) levels are greater than or equal to. Try to Limit the number of entires unless it could lead to a worse answer. Make sure to select only the column required for the query.
 

vitals

 


CREATE TABLE vitals (
	"index" INTEGER, 
	id INTEGER, 
	"Age" INTEGER, 
	"Bgr" INTEGER, 
	"Al" INTEGER, 
	"Bu" INTEGER, 
	"Sc" REAL, 
	"Sod" INTEGER, 
	"Pot" REAL, 
	"Hemo" REAL, 
	"Pcv" REAL, 
	"Wc" REAL, 
	"Rc" REAL, 
	"Htn" TEXT, 
	"Dm" TEXT, 
	"Cad" TEXT, 
	"Pe" TEXT, 
	"Ane" TEXT, 
	"Chloride" REAL, 
	"BiCarbonate" INTEGER, 
	date TEXT
)

/*
3 rows from vitals table:
index	id	Age	Bgr	Al	Bu	Sc	Sod	Pot	Hemo	Pcv	Wc	Rc	Htn	Dm	

In [ ]:
print(result)

**Your Hemoglobin Readings Report**

**Summary of Your Hemoglobin Readings:**
------------------------------------------

* Your highest hemoglobin reading was **10.5**, recorded on **14-Jan-22**.
* This reading is below the threshold for anemia (13.5 g/dL or higher), so it does not indicate anemia.

I hope this report helps you understand your hemoglobin readings!
